## Prog 3: 0915 Hrs Nifty_Total Status with LTP
## Sheet 2 : Pchange comparison timestamp(tbdone)
## Sheet 3: Ranking system (To be corrected)

In [ ]:
import requests
import pandas as pd
from datetime import datetime, time as dt_time
import schedule
import time
import openpyxl
import os
from openpyxl.styles import Font, PatternFill

# Initialize session object
session = requests.Session()
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
}

def get_cookie():
    url = 'https://www.nseindia.com'
    response = session.get(url, headers=headers)
    cookies = response.cookies.get_dict()
    cookie_str = '; '.join([f"{key}={value}" for key, value in cookies.items()])
    return cookie_str

def fetch_nifty_data():
    if 'cookie' not in headers or headers['cookie'] is None:
        headers['cookie'] = get_cookie()
    try:
        response = session.get('https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20TOTAL%20MARKET', headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        if response.status_code == 401:
            print("Cookie expired. Fetching a new cookie...")
            headers['cookie'] = get_cookie()
            return fetch_nifty_data()
        else:
            print(f"HTTP Error {response.status_code}: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
    return None

def clean_value(value):
    if isinstance(value, str):
        return float(value.replace(',', ''))
    return float(value)

def extract_data(positions):
    data_list = []
    for item in positions['data']:
        symbol = item['symbol']
        ltp = clean_value(item['lastPrice'])
        p_change = clean_value(item['pChange'])
        open_price = clean_value(item['open'])
        day_low = clean_value(item['dayLow'])
        day_high = clean_value(item['dayHigh'])
        previous_close = clean_value(item['previousClose'])
        data_list.append([symbol, ltp, p_change, open_price, day_low, day_high, previous_close])
    df = pd.DataFrame(data_list, columns=['symbol', 'LTP', 'pChange', 'Open', 'D_Low', 'D_High', 'Pr.Close'])
    return df

def append_timestamp_columns(existing_df, new_df):
    timestamp = datetime.now().strftime('%H_%M_%S')
    existing_df[f'LTP_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['LTP'])
    existing_df[f'pChange_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['pChange'])
    return existing_df

def calculate_pchange_diff(existing_df, timestamp):
    pchange_columns = [col for col in existing_df.columns if col.startswith('pChange_') and col != f'pChange_{timestamp}']
    
    if pchange_columns:
        pchange_columns.sort()
        previous_pchange_col = pchange_columns[-1]
        existing_df[f'Diff_{timestamp}'] = existing_df[f'pChange_{timestamp}'] - existing_df[previous_pchange_col]
    
    return existing_df

def apply_styles(ws):
    # Apply styles to the header
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    # Adjust column widths
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Apply filter to the first row
    ws.auto_filter.ref = ws.dimensions

    # Freeze the header row and the first column
    ws.freeze_panes = ws['B2']  # This will freeze the first row and the first column

def save_to_csv(df, filename):
    if os.path.exists(filename):
        existing_df = pd.read_csv(filename)
        timestamp = datetime.now().strftime('%H_%M_%S')
        existing_df = append_timestamp_columns(existing_df, df)
        existing_df = calculate_pchange_diff(existing_df, timestamp)
    else:
        existing_df = df

    existing_df.to_csv(filename, index=False)
    print(f"Data updated in {filename} at {datetime.now().strftime('%d%b_%H:%M:%S')}")

def update_filtered_data_csv(df, filename, filter_condition):
    if os.path.exists(filename):
        existing_df = pd.read_csv(filename)
        timestamp = datetime.now().strftime('%H_%M_%S')
        filtered_df = df[filter_condition]
        existing_df = append_timestamp_columns(existing_df, filtered_df)
        existing_df = calculate_pchange_diff(existing_df, timestamp)
    else:
        existing_df = df[filter_condition]

    existing_df.to_csv(filename, index=False)
    # print(f"Filtered data updated in {filename}")

def merge_csv_files_to_excel(csv_files, excel_filename, sheet_names):
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        for csv_file, sheet_name in zip(csv_files, sheet_names):
            df = pd.read_csv(csv_file)
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            apply_styles(writer.sheets[sheet_name])
            apply_color_coding(writer.sheets[sheet_name])
            # print(f"Data from {csv_file} saved to {excel_filename} in sheet {sheet_name}")

def apply_color_coding(ws):
    pchange_columns = [cell.column for cell in ws[1] if cell.value and cell.value.startswith('pChange')]
    
    for col in pchange_columns:
        values = []
        for cell in ws.iter_cols(min_col=col, max_col=col, min_row=2):
            try:
                values.append(float(cell[0].value))
            except (TypeError, ValueError):
                continue
        if not values:
            continue
        
        min_value, max_value = min(values), max(values)
        range_value = max_value - min_value if max_value != min_value else 1
        
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col, max_col=col):
            for cell in row:
                try:
                    value = float(cell.value)
                    normalized_value = (value - min_value) / range_value
                    if value > 0:
                        shade = int(204 + 51 * normalized_value)
                        hex_color = f"{shade:02X}FF{shade:02X}"  # Green shades
                        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
                    elif value < 0:
                        shade = int(204 + 51 * abs(normalized_value))
                        hex_color = f"FF{shade:02X}{shade:02X}"  # Red shades
                        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
                except (TypeError, ValueError):
                    continue

def job():
    current_time = datetime.now().time()
    if ((current_time >= dt_time(0, 15) and current_time <= dt_time(9, 40) and current_time.minute % 1 == 0) or 
        (current_time > dt_time(9, 40) and current_time < dt_time(10, 45) and current_time.minute % 3 == 0) or 
        (current_time > dt_time(10, 45) and current_time < dt_time(14, 0) and current_time.minute % 1 == 0) or 
        (current_time >= dt_time(14, 0) and current_time <= dt_time(15, 30) and current_time.minute % 1 == 0)):

        nifty_data = fetch_nifty_data()
        if nifty_data:
            df = extract_data(nifty_data)
            # print(df.head())

            # Save individual CSV files
            csv_files = ['6_nifty_total_5_min.csv', '6_Open_Eql_Low.csv', '6_Open_Pr.Close.csv', '6_O_L_Pr.Close.csv', '6_Open_D_High.csv']
            sheet_names = ['Nifty Data', 'O=L', 'O>Pr.Close', 'O=L<Pr.Close', 'O=D_High']
            save_to_csv(df, csv_files[0])
            update_filtered_data_csv(df, csv_files[1], df['Open'] == df['D_Low'])
            update_filtered_data_csv(df, csv_files[2], df['Open'] > df['Pr.Close'])
            update_filtered_data_csv(df, csv_files[3], (df['Open'] == df['D_Low']) & (df['Open'] > df['Pr.Close']))
            update_filtered_data_csv(df, csv_files[4], df['Open'] == df['D_High'])

            # Merge CSV files into one Excel workbook with different sheets
            merge_csv_files_to_excel(csv_files, '6_Nifty_total_csv_to_excel_5_min.xlsx', sheet_names)

# Schedule the job every 30 seconds
schedule.every(30).seconds.do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)
